In [1]:
# use finrl
import pandas as pd
import numpy as np
import datetime
import yfinance as yf
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools

In [2]:
INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [1]:
from tvDatafeed import TvDatafeed,Interval
tv = TvDatafeed(username=None,password=None)

you are using nologin method, data you access may be limited


In [2]:
df = tv.get_hist("SCB", "SET", interval=Interval.in_daily, n_bars=365)
df = df.reset_index()

In [3]:
df

,datetime,symbol,open,high,low,close,volume
0,2022-02-08 09:00:00,SET:SCB,127.0,130.0,127.0,130.0,20279900.0
1,2022-02-09 09:00:00,SET:SCB,130.5,134.5,130.5,134.5,38519400.0
2,2022-02-10 09:00:00,SET:SCB,135.0,136.0,134.0,135.5,26318600.0
3,2022-02-11 09:00:00,SET:SCB,134.0,138.0,133.5,137.0,30193600.0
4,2022-02-14 09:00:00,SET:SCB,134.0,134.5,132.0,134.0,17316600.0
...,...,...,...,...,...,...,...
360,2023-08-16 09:00:00,SET:SCB,112.5,113.5,112.0,113.5,6727093.0
361,2023-08-17 09:00:00,SET:SCB,112.5,114.0,112.0,114.0,7541616.0
362,2023-08-18 09:00:00,SET:SCB,113.5,114.0,113.0,113.5,10165096.0
363,2023-08-21 09:00:00,SET:SCB,113.5,114.5,113.0,113.5,6232917.0


In [39]:
df = df.rename(columns = {"datetime":"date","symbol":"tic"})

In [40]:
df['date'] = df['date'].dt.date.astype(str)

In [45]:
df['tic'] = "SCB"

In [42]:
def convertday2int(text):
    date = datetime.datetime.strptime(text, '%Y-%m-%d').date()
    weekday = (date.weekday()) % 5
    return weekday


In [43]:
df['day'] = df['date'].apply(convertday2int)

In [49]:
df.head()

,date,tic,open,high,low,close,volume,symbol,day
0,2021-10-15,SCB,123.5,124.5,122.5,123.0,14625900.0,SCB,4
1,2021-10-18,SCB,123.0,124.5,123.0,123.5,9755000.0,SCB,0
2,2021-10-19,SCB,124.0,124.5,122.0,122.5,7837900.0,SCB,1
3,2021-10-20,SCB,122.5,126.0,122.0,125.5,20217400.0,SCB,2
4,2021-10-21,SCB,126.0,126.5,124.0,124.0,11767600.0,SCB,3


In [47]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df)
     

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (383, 8)
Successfully added vix
Successfully added turbulence index


In [50]:
processed.head()

,date,tic,open,high,low,close,volume,symbol,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2021-10-15,SCB,123.5,124.5,122.5,123.0,14625900.0,SCB,4,0.000000,123.957107,122.542893,100.000000,66.666667,100.000000,123.000,123.000,16.299999,0.0
1,2021-10-18,SCB,123.0,124.5,123.0,123.5,9755000.0,SCB,0,0.011218,123.957107,122.542893,100.000000,66.666667,100.000000,123.250,123.250,16.309999,0.0
2,2021-10-19,SCB,124.0,124.5,122.0,122.5,7837900.0,SCB,1,-0.016716,124.000000,122.000000,32.584270,-100.000000,100.000000,123.000,123.000,15.700000,0.0
3,2021-10-20,SCB,122.5,126.0,122.0,125.5,20217400.0,SCB,2,0.079303,126.254956,120.995044,78.198221,127.272727,23.178808,123.625,123.625,15.490000,0.0
4,2021-10-21,SCB,126.0,126.5,124.0,124.0,11767600.0,SCB,3,0.070120,126.002173,121.397827,57.925934,100.694444,37.003745,123.700,123.700,15.010000,0.0
